Dropped selected columns by guidance of Dr.Mahesh and Team

In [ ]:
columns_to_drop = ['form', 'version', 'country', 'centre', 'school', 'serial','seriala1','seriala2','dint','dint_stata','age','agecalc','dbirth',
                   'dbirth_stata','wgtunit','weight_kg','hgtunit','height_m','language','asthplan','school12','rash12','sitesev','rashage',
                   'rclear12','rawake12','eczemaev','eczedoc','animoth','bweight_kg','bwgtunit','milkyngb','annualpm25_poll']

# Drop the specified columns
data_new = data_new.drop(columns=columns_to_drop)

# Now 'data' will have the specified columns dropped
print(data_new.info())
data_new.head()

Concentrating on a subset of the features, we try to convert the categorical data</br> into numerical data using pd.to_numerical and few of the attributes gets converted into the numerical data

data_conv[['chflr01','chflr02','chflr03','chflr04']]

In [ ]:
# get the list of columns which are stored as strings but when i see it manually, it is numerical. 

import pandas as pd

# Assuming 'data_con' is your DataFrame
numerical_columns_stored_as_strings = []

# Iterate over each column
for col in data_conv.columns:
    # Check if the column is of object type
    if data_conv[col].dtype == 'object':
        # Attempt to convert values to numeric
        try:
            pd.to_numeric(data_conv[col])
            # If conversion is successful, add the column to the list
            numerical_columns_stored_as_strings.append(col)
            print(f'its conversion happened & its unique values are:{col} {print(data_conv[col].unique())} \n \n')
        except ValueError:
            print(f'{col}:value_error')
            print(data_conv[col].unique())
            print('\n\n')
            # If conversion fails, continue to the next column
            pass

print("Columns with numerical values stored as strings:")
print(numerical_columns_stored_as_strings)


CONCLUSION

1. Custom transformation first: Split the composite feature into multiple sub-features.
2. Imputation: Handle missing values in a meaningful way for each sub-feature.

REASON

because the columns (like chflr01) are composite features. Performing custom transformations first will give you better control over how to handle missing values for each sub-feature individually

# 1. Object Columns

bweight

In [ ]:
# print both numeric & non-numeric values

import pandas as pd

# Assuming data_conv is your DataFrame
# Filter numeric values
numeric_values = data_conv[data_conv['bweight'].str.isnumeric() & (data_conv['bweight'] != '')]['bweight']

# Filter non-numeric values
non_numeric_values = data_conv[~data_conv['bweight'].str.isnumeric() | (data_conv['bweight'] == '')]['bweight']

# Display numeric values
print("Numeric values in 'bweight' column:")
print(numeric_values)

# Display non-numeric values
print("\nNon-numeric values in 'bweight' column:")
print(non_numeric_values)

# Count unique elements in numeric values
numeric_unique_count = numeric_values.unique()
print("\n \n unique numeric elements", numeric_unique_count)

# Count unique elements in non-numeric values
non_numeric_unique_count = non_numeric_values.nunique()

# Display the number of unique elements in each group
print("Number of unique elements in numeric values:", len(numeric_unique_count))
print("Number of unique elements in non-numeric values:", non_numeric_unique_count)


We can observe that there are decimal values as entries and they are not numerical type.</br> hence we can convert them into float type by first including the decimal values into th numerical attribute using regex </br>

In [ ]:
import pandas as pd
import numpy as np

# Assuming data_conv_dtype is your DataFrame
# Replace empty strings with NaN
data_conv_dtype['bweight'] = data_conv_dtype['bweight'].replace('', np.nan)

# Filter out non-numeric values or empty strings
non_numeric_values = data_conv_dtype.loc[~data_conv_dtype['bweight'].astype(str).str.match(r'^-?\d+\.?\d*$'), 'bweight']

# Display non-numeric values
print("Non-numeric values or empty strings in 'bweight' column:")
print(non_numeric_values)

We Found some inconsistancies in the entry hence resolving them manually and converting the column into the float type.

In [ ]:
import pandas as pd
import numpy as np

# Assuming data_conv_dtype is your DataFrame
# Replace empty strings with NaN
data_conv_dtype['bweight'] = data_conv_dtype['bweight'].replace('', np.nan)

# Replace '2..6' with 2.6
data_conv_dtype['bweight'] = data_conv_dtype['bweight'].replace('2..6', '2.6')

# Replace '.5' with 0.5
data_conv_dtype['bweight'] = data_conv_dtype['bweight'].replace('.5', '0.5')

# Convert 'bweight' column to float dtype
data_conv_dtype['bweight'] = data_conv_dtype['bweight'].astype(float)

# Now 'bweight' column should be of float dtype with corrected values


now lets try to encode chflr01, chflr02, chflr03, chflr04</br>

chflr01 is a column in a dataframe named data_conv_dtype. it has 2730 rows.</br>
chflr01 is of object dtype. say for example one of the row values of chflr01='1222'.</br>
here 4 numbers indicate 4 type of values which are not shown in the dataframe. </br>

i want to encode each digit in the chflr01 column to 4 different new columns value viz., </br>

    - chflr01_never, 
    chflr01_at this time, 
    chflr01_during first year of this child ,
    chflr01_at some time .
</br>
encoding should happen in the below format:</br>
if chflr01='1222' then the values in the new column values are</br>

    chflr01_never = 1
    chflr01_at this time = 0
    chflr01_during first year of this child = 0
    chflr01_at some time = 0
</br>
there are only 2 possible numbers 1, 2 where 1 indicates yes which should be encoded as 1 and 2 indicates no which should be </br>encoded as 0
</br>

so this is kind of a custom transformation

In [ ]:
columns_to_process = ['chflr01','chflr02', 'chflr03', 'chflr04']
data_test[columns_to_process].head(10)

In [ ]:
for column in columns_to_process:
    unique_values = data_test[column].unique()
    print(f"Unique values in column '{column}':")
    print(unique_values)
    print()

The below code, Does One-hot encoding in a custom way.</br>
At the end it drops the original cloumns : ['chflr01','chflr02', 'chflr03', 'chflr04']

Resulting in extra 4*4 columns and dropping 4 columns = 161 columns from 149 columns

In [ ]:
import pandas as pd

# Columns to process
columns_to_process = ['chflr01','chflr02', 'chflr03', 'chflr04']

for col in columns_to_process:
    # Initialize new columns with default value of 0
    data_test[f'{col}_never'] = 0
    data_test[f'{col}_at_this_time'] = 0
    data_test[f'{col}_during_first_year_of_this_child'] = 0
    data_test[f'{col}_at_some_time'] = 0

    # Iterate over each row in the column
    for index, row in data_test.iterrows():
        col_value = row[col]

        # Assign values to new columns based on the column value
        for i, digit in enumerate(col_value):
            if i == 0:
                data_test.at[index, f'{col}_never'] = 1 if digit == '1' else 0
            elif i == 1:
                data_test.at[index, f'{col}_at_this_time'] = 1 if digit == '1' else 0
            elif i == 2:
                data_test.at[index, f'{col}_during_first_year_of_this_child'] = 1 if digit == '1' else 0
            elif i == 3:
                data_test.at[index, f'{col}_at_some_time'] = 1 if digit == '1' else 0

    # Drop the original column
    data_test.drop(col, axis=1, inplace=True)

# Verify the changes
data_test.head()

['chflr01_never', 'chflr01_at_this_time', 'chflr01_during_first_year_of_this_child', 'chflr01_at_some_time', 'chflr02_never', 'chflr02_at_this_time', 'chflr02_during_first_year_of_this_child', 'chflr02_at_some_time', 'chflr03_never', 'chflr03_at_this_time', 'chflr03_during_first_year_of_this_child', 'chflr03_at_some_time', 'chflr04_never', 'chflr04_at_this_time', 'chflr04_during_first_year_of_this_child', 'chflr04_at_some_time']

To standardize the unique values in the columns (medpil1a, medpil2a, medpil3a, medpil4a), you can follow the process outlined below:

Convert to lowercase: Convert all values in the columns to lowercase to handle different cases (uppercase, lowercase, or mixed).

Remove whitespace: Strip leading and trailing whitespace to handle accidental spaces.

Split composite entries: For entries containing multiple medications separated by slashes (/), you can split them into individual components and handle each component.

Create a mapping: Create a mapping from original values to standardized values, particularly for common misspellings, abbreviations, or synonyms. Use this mapping to replace variations of the same medication with a standard representation.

Apply the mapping: Replace values in the columns with their standardized equivalents using the mapping.


Unique values in column 'medpil1a':
['9' '1' '' 'Asthaline' 'ASTHALIN' 'asthaline' 'Asthalin' 'Deriphiline'
 'Deriphyline' 'Theoasthalin' 'Derifilin'
 'Asthalin/ Theoasthalin/ Deriphylline/ Doxophylline/ Abphylline'
 'asthalin']

Unique values in column 'medpil2a':
['9' 'Montear' '1' '2' '' 'MONTEK' 'Montek' '.'
 'Montair/ Singulair/ Montek/ Telekast/ Montek LC' 'Montair' 'montek'
 'MONTAIR']

Unique values in column 'medpil3a':
['9' '1' '2' '' 'Dephcarte' 'Omnacortil' '.'
 'Wysolone/ Defcorte/ Wymesone/ Omnacortil/ Decadran' 'Visolin' 'defcort'
 'WYSOLONE']

Unique values in column 'medpil4a':
['9' '1' '2' '' 'Azee' 'Azithral' 'Augmentin' '.'
 'Advent/ Augmentin/ Azithral/ Azee/ Cephalexin' 'Advant' 'advent'
 'Azee 3 days course' 'ADVENT']

In [ ]:
import pandas as pd

# Assuming data_medpill dataframe is already present

# Define the columns to process
columns_to_process = ['medpil1a', 'medpil2a', 'medpil3a', 'medpil4a']

# Define a mapping dictionary to standardize spelling and variations
# Each key is a variation or spelling error, and each value is the desired standardized form
standardization_dict = {
    # Standardize 'medpil1a'
    'ASTHALIN': 'Asthaline',
    'asthaline': 'Asthaline',
    'Asthalin': 'Asthaline',
    'Deriphiline': 'Deriphyline',
    'Deriphyline': 'Deriphyline',
    'Derifilin': 'Deriphyline',
    'Asthalin/ Theoasthalin/ Deriphylline/ Doxophylline/ Abphylline': 'Asthaline',
    'asthalin': 'Asthaline',
    
    # Standardize 'medpil2a'
    'MONTEK': 'Montek',
    'montek': 'Montek',
    'Montear': 'Montair',
    'montear': 'Montair',
    'MONTAIR': 'Montair',
    'Montair': 'Montair',
    'Montair/ Singulair/ Montek/ Telekast/ Montek LC': 'Montair',
    
    # Standardize 'medpil3a'
    'Dephcarte': 'Defcorte',
    'defcort': 'Defcorte',
    'Omnacortil': 'Omnacortil',
    'Wysolone/ Defcorte/ Wymesone/ Omnacortil/ Decadran': 'Wysolone',
    'Visolin': 'Wysolone',
    'visolin': 'Wysolone',
    'WYSOLONE': 'Wysolone',
    
    # Standardize 'medpil4a'
    'Azithral': 'Azithral',
    'Azee': 'Azithral',
    'Augmentin': 'Augmentin',
    'advent': 'Advent',
    'Advant': 'Advent',
    'ADVENT': 'Advent',
    'Advent/ Augmentin/ Azithral/ Azee/ Cephalexin' : 'Azithral',
    'Azee 3 days course': 'Azithral',
}

# Standardization function
def standardize_column(series, medication):
    # Replace '1' with the specific medication
    series = series.replace('1', medication)
    
    # Split composite entries (separated by '/'), keep only the first medication
    series = series.apply(lambda x: x.split('/')[0].strip() if '/' in x else x)
    
    # Map variations to a standard form
    series = series.replace(standardization_dict)
    
    # Convert to lowercase and strip leading/trailing whitespace
    series = series.str.lower().str.strip()
    
    return series

# Apply the standardization function to each column in columns_to_process
medication_mapping = {
    'medpil1a': 'Asthaline',
    'medpil2a': 'Montek',
    'medpil3a': 'Defcorte',
    'medpil4a': 'Azithral'
}

for col in columns_to_process:
    data_medpill[col] = standardize_column(data_medpill[col], medication_mapping[col])

# Now, data_medpill contains standardized values in the columns_to_process


Checking for the patterns occurig between medphill and med{col}

In [ ]:
# List of columns of interest
columns_of_interest = ['medpil1b', 'medpil2b', 'medpil3b', 'medpil4b']

# Loop over each column to check for NaN and get the value counts of 'medpill'
for col in columns_of_interest:
    print(f"Value counts in 'medpill' when {col} is NaN:")
    value_counts = data_medpill[data_medpill[col].isna()]['medpill'].value_counts()
    print(value_counts)
    print()  # Blank line for readability

Value counts in 'medpill' when medpil1b is NaN:</br>
medpill</br>
No     24</br>
Yes     3</br>
Name: count, dtype: int64</br>

Value counts in 'medpill' when medpil2b is NaN:</br>
medpill</br>
No     24</br>
Yes    22</br>
Name: count, dtype: int64</br>
</br>
Value counts in 'medpill' when medpil3b is NaN:</br>
medpill</br>
No     24</br>
Yes    22</br>
Name: count, dtype: int64</br>
</br>
Value counts in 'medpill' when medpil4b is NaN:</br>
medpill</br>
No     24</br>
Yes    21</br>
Name: count, dtype: int64</br>

In [ ]:
import pandas as pd

# Assuming data_test is your dataframe

# Filter the dataframe where 'medpill' is Yes
filtered_data = data_medpill[data_medpill['medpill'] == 'Yes']

# Get the unique values and their value counts for each of the specified columns
for column in ['medpil1b', 'medpil2b', 'medpil3b', 'medpil4b']:
    print(f"Unique values and their counts in column '{column}' when 'medpill' is Yes:")
    value_counts = filtered_data[column].value_counts(dropna=False)
    print(value_counts)
    print()

Unique values and their counts in column 'medpil1b' when 'medpill' is Yes:</br>
medpil1b</br>
Only when needed    17</br>
In short courses     5</br>
NaN                  3</br>
Name: count, dtype: int64</br>
</br>
Unique values and their counts in column 'medpil2b' when 'medpill' is Yes:</br>
medpil2b</br>
NaN                 22</br>
Only when needed     2</br>
In short courses     1</br>
Name: count, dtype: int64</br>
</br>
Unique values and their counts in column 'medpil3b' when 'medpill' is Yes:</br>
medpil3b</br>
NaN                 22</br>
Only when needed     3</br>
Name: count, dtype: int64</br>
</br>
Unique values and their counts in column 'medpil4b' when 'medpill' is Yes:</br>
medpil4b</br>
NaN                 21</br>
Only when needed     4</br>
Name: count, dtype: int64</br>

Conclusion:-
1. 
x = ['medpil1a', 'medpil2a', 'medpil3a', 'medpil4a']
y = ['medpil1b', 'medpil2b', 'medpil3b', 'medpil4b']

Case 1. when medpill = NaN

        X--->  encode (9=''=.)----replaceby--> = NaN <----- X (or 0 <---X)
        Y---> keep NaN(will be replaced during encoding) <---- Y   (or replace by 0 is also a possibility)

Case 2. When medpill = No

        X---> encode (9,'',.) in X by 0 <---X
        Y---> keep NaN(will be replaced during encoding)<---- Y  (or replace by 0 <---- Y)

Case 3. When medpill = Yes

        X ---> (9, '', .) --(if)--> medpil1a = "Bronchodialators" or "Asthaline"
                          --(if)--> medpil2a = "Antihistamine" or "Montair"
                          --(if)--> medpil3a = "Corticosteroids" or "Wysolone"
                          --(if)--> medpil4a = "Antibiotics" or "Azithral"
        Y ---> NaN ----(replace by)----> 'only when needed'<---- Y
                    [       (or)                  
                   ----(replace by)----> 'replace during encoding' ]

case 1

In [ ]:
import numpy as np

# Define the columns to clean
columns_to_clean = ['medpil1a', 'medpil2a', 'medpil3a', 'medpil4a']

# Apply the replacement only where medpill is NaN and ensure values are treated as strings
data_medpill.loc[data_medpill['medpill'].isna(), columns_to_clean] = \
    data_medpill.loc[data_medpill['medpill'].isna(), columns_to_clean].replace(
        {'9': np.nan, '': np.nan, '.': np.nan}, 
        regex=False
    )


case 2

In [ ]:
import numpy as np

# Define the columns for each operation
columns_to_clean = ['medpil1a', 'medpil2a', 'medpil3a', 'medpil4a']
columns_of_interest = ['medpil1b', 'medpil2b', 'medpil3b', 'medpil4b']

# Task 1: Encode '9', '', '.' with NaN in columns_to_clean when medpill = 'No'
# Use .loc to target the rows where medpill is 'No' and replace the values
data_medpill.loc[data_medpill['medpill'] == 'No', columns_to_clean] = \
    data_medpill.loc[data_medpill['medpill'] == 'No', columns_to_clean].replace(['9', '', '.'], np.nan)

# Task 2: Convert columns_of_interest to numeric and replace NaN with 0 when medpill = 'No'
for col in columns_of_interest:
    # Convert the column to a numeric type, coercing errors to NaN
    data_medpill[col] = pd.to_numeric(data_medpill[col], errors='coerce')
    # Replace NaN with 0 where medpill is 'No'
    data_medpill.loc[data_medpill['medpill'] == 'No', col] = \
        data_medpill.loc[data_medpill['medpill'] == 'No', col].fillna(0)

# Verify the changes by printing unique values after the transformations
print("Unique values after replacement for columns_to_clean when medpill = 'No':")
for col in columns_to_clean:
    print(f"Unique values in column '{col}' (when medpill = 'No'):")
    print(data_medpill.loc[data_medpill['medpill'] == 'No', col].unique())
    print()  # Blank line for readability

print("Unique values after replacement for columns_of_interest when medpill = 'No':")
for col in columns_of_interest:
    print(f"Unique values in column '{col}'(when medpill = 'No'):")
    print(data_medpill.loc[data_medpill['medpill'] == 'No', col].unique())
    print()  # Blank line for readability


Case 3

In [ ]:
import numpy as np

# Define the values to replace and their corresponding replacements
replacement_map = {
    'medpil1a': 'bronchodialators',
    'medpil2a': 'antihistamine',
    'medpil3a': 'corticosteroids',
    'medpil4a': 'antibiotics'
}

# Define the values to be replaced
values_to_replace = ['9', '', '.', '2']

# Define the columns for each operation
columns_to_clean = ['medpil1a', 'medpil2a', 'medpil3a', 'medpil4a']
columns_of_interest = ['medpil1b', 'medpil2b', 'medpil3b', 'medpil4b']

# Task 1: Replace specified values in columns_to_clean when medpill = 'Yes'
for column, replacement in replacement_map.items():
    print(column, replacement)
    # Replace values directly without using replace function
    data_medpill.loc[data_medpill['medpill'] == 'Yes', column] = \
        np.where(data_medpill.loc[data_medpill['medpill'] == 'Yes', column].isin(values_to_replace), 
                 replacement, 
                 data_medpill.loc[data_medpill['medpill'] == 'Yes', column])
    
    print(data_medpill.loc[data_medpill['medpill'] == 'Yes', column].unique())
    print() 

# Task 2: Replace NaN in columns_of_interest with 'only when needed' when medpill = 'Yes'
for col in columns_of_interest:
    data_medpill.loc[data_medpill['medpill'] == 'Yes', col] = \
        data_medpill.loc[data_medpill['medpill'] == 'Yes', col].fillna('only when needed')

# Verify the changes by printing unique values after the transformations
print("Unique values after replacement for columns_to_clean when medpill = 'Yes':")
for col in columns_to_clean:
    print(f"Unique values in column '{col}' (when medpill = 'Yes'):")
    print(data_medpill.loc[data_medpill['medpill'] == 'Yes', col].unique())
    print()  # Blank line for readability

print("Unique values after replacement for columns_of_interest when medpill = 'Yes':")
for col in columns_of_interest:
    print(f"Unique values in column '{col}' (when medpill = 'Yes'):")
    print(data_medpill.loc[data_medpill['medpill'] == 'Yes', col].unique())
    print()  # Blank line for readability


Unique values and their counts in column 'medpil1a' when 'medpill' is Yes:</br>
medpil1a</br>
asthaline           18</br>
bronchodialators     3</br>
deriphyline          3</br>
theoasthalin         1</br>
Name: count, dtype: int64</br>
</br>
Unique values and their counts in column 'medpil2a' when 'medpill' is Yes:</br>
medpil2a</br>
antihistamine    22</br>
montek            3</br>
Name: count, dtype: int64</br>
</br>
Unique values and their counts in column 'medpil3a' when 'medpill' is Yes:</br>
medpil3a</br>
corticosteroids    23</br>
defcorte            1</br>
omnacortil          1</br>
Name: count, dtype: int64</br>
</br>
Unique values and their counts in column 'medpil4a' when 'medpill' is Yes:</br>
medpil4a</br>
antibiotics    22</br>
azithral        2</br>
augmentin       1</br>
Name: count, dtype: int64</br>

In [ ]:
import pandas as pd

print("value counts in medpill")
value_counts = data_medpill['medpill'].value_counts(dropna = False)
print(value_counts)
print("_"*100)
print()

print("value counts in X and Y")
# Get the unique values and their value counts for each of the specified columns
for column in ['medpil1a', 'medpil2a', 'medpil3a', 'medpil4a']:
    #print(f"Unique values and their counts in column '{column}' when 'medpill' is NaN:")
    value_counts = data_medpill[column].value_counts(dropna=False)
    print(value_counts)
    print()

for column in ['medpil1b', 'medpil2b', 'medpil3b', 'medpil4b']:
    #print(f"Unique values and their counts in column '{column}' when 'medpill' is NaN:")
    value_counts = data_medpill[column].value_counts(dropna=False)
    print(value_counts)
    print()


print("_"*100)
print()

print("value counts in X and Y under different conditions of medpill")
print()
# Filter the dataframe where 'medpill' is Yes
filtered_data = data_medpill[data_medpill['medpill'] == 'Yes']

# Get the unique values and their value counts for each of the specified columns

for column in ['medpil1a', 'medpil2a', 'medpil3a', 'medpil4a']:
    print(f"Unique values and their counts in column '{column}' when 'medpill' is Yes:")
    value_counts = filtered_data[column].value_counts(dropna=False)
    print(value_counts)
    print()

print("-"*50)
for column in ['medpil1b', 'medpil2b', 'medpil3b', 'medpil4b']:
    print(f"Unique values and their counts in column '{column}' when 'medpill' is Yes:")
    value_counts = filtered_data[column].value_counts(dropna=False)
    print(value_counts)
    print()

print("_"*100)

# Filter the dataframe where 'medpill' is Yes
filtered_data = data_medpill[data_medpill['medpill'] == 'No']

# Get the unique values and their value counts for each of the specified columns
for column in ['medpil1a', 'medpil2a', 'medpil3a', 'medpil4a']:
    print(f"Unique values and their counts in column '{column}' when 'medpill' is No:")
    value_counts = filtered_data[column].value_counts(dropna=False)
    print(value_counts)
    print()
print("-"*50)

for column in ['medpil1b', 'medpil2b', 'medpil3b', 'medpil4b']:
    print(f"Unique values and their counts in column '{column}' when 'medpill' is No:")
    value_counts = filtered_data[column].value_counts(dropna=False)
    print(value_counts)
    print()


print("_"*100)

# Filter the dataframe where 'medpill' is NaN
filtered_data = data_medpill[data_medpill['medpill'].isna()]

# Get the unique values and their value counts for each of the specified columns
for column in ['medpil1a', 'medpil2a', 'medpil3a', 'medpil4a']:
    print(f"Unique values and their counts in column '{column}' when 'medpill' is NaN:")
    value_counts = filtered_data[column].value_counts(dropna=False)
    print(value_counts)
    print()

print("-"*50)
for column in ['medpil1b', 'medpil2b', 'medpil3b', 'medpil4b']:
    print(f"Unique values and their counts in column '{column}' when 'medpill' is NaN:")
    value_counts = filtered_data[column].value_counts(dropna=False)
    print(value_counts)
    print()

print("_"*100)

value counts in medpill
medpill
NaN    2681
Yes      25
No       24
Name: count, dtype: int64
____________________________________________________________________________________________________

value counts in X and Y
medpil1a
NaN                 2705
asthaline             18
bronchodialators       3
deriphyline            3
theoasthalin           1
Name: count, dtype: int64

medpil2a
NaN              2705
antihistamine      22
montek              3
Name: count, dtype: int64

medpil3a
NaN                2705
corticosteroids      23
defcorte              1
omnacortil            1
Name: count, dtype: int64

medpil4a
NaN            2705
antibiotics      22
azithral          2
augmentin         1
Name: count, dtype: int64

medpil1b
NaN                 2681
only when needed      25
0.0                   24
Name: count, dtype: int64

medpil2b
NaN                 2681
only when needed      25
0.0                   24
Name: count, dtype: int64

medpil3b
NaN                 2681
only when needed      25
0.0                   24
Name: count, dtype: int64

medpil4b
NaN                 2681
only when needed      25
0.0                   24
Name: count, dtype: int64

____________________________________________________________________________________________________

value counts in X and Y under different conditions of medpill

Unique values and their counts in column 'medpil1a' when 'medpill' is Yes:
medpil1a
asthaline           18
bronchodialators     3
deriphyline          3
theoasthalin         1
Name: count, dtype: int64

Unique values and their counts in column 'medpil2a' when 'medpill' is Yes:
medpil2a
antihistamine    22
montek            3
Name: count, dtype: int64

Unique values and their counts in column 'medpil3a' when 'medpill' is Yes:
medpil3a
corticosteroids    23
defcorte            1
omnacortil          1
Name: count, dtype: int64

Unique values and their counts in column 'medpil4a' when 'medpill' is Yes:
medpil4a
antibiotics    22
azithral        2
augmentin       1
Name: count, dtype: int64

--------------------------------------------------
Unique values and their counts in column 'medpil1b' when 'medpill' is Yes:
medpil1b
only when needed    25
Name: count, dtype: int64

Unique values and their counts in column 'medpil2b' when 'medpill' is Yes:
medpil2b
only when needed    25
Name: count, dtype: int64

Unique values and their counts in column 'medpil3b' when 'medpill' is Yes:
medpil3b
only when needed    25
Name: count, dtype: int64

Unique values and their counts in column 'medpil4b' when 'medpill' is Yes:
medpil4b
only when needed    25
Name: count, dtype: int64

____________________________________________________________________________________________________
Unique values and their counts in column 'medpil1a' when 'medpill' is No:
medpil1a
NaN    24
Name: count, dtype: int64

Unique values and their counts in column 'medpil2a' when 'medpill' is No:
medpil2a
NaN    24
Name: count, dtype: int64

Unique values and their counts in column 'medpil3a' when 'medpill' is No:
medpil3a
NaN    24
Name: count, dtype: int64

Unique values and their counts in column 'medpil4a' when 'medpill' is No:
medpil4a
NaN    24
Name: count, dtype: int64

--------------------------------------------------
Unique values and their counts in column 'medpil1b' when 'medpill' is No:
medpil1b
0.0    24
Name: count, dtype: int64

Unique values and their counts in column 'medpil2b' when 'medpill' is No:
medpil2b
0.0    24
Name: count, dtype: int64

Unique values and their counts in column 'medpil3b' when 'medpill' is No:
medpil3b
0.0    24
Name: count, dtype: int64

Unique values and their counts in column 'medpil4b' when 'medpill' is No:
medpil4b
0.0    24
Name: count, dtype: int64

____________________________________________________________________________________________________
Unique values and their counts in column 'medpil1a' when 'medpill' is NaN:
medpil1a
NaN    2681
Name: count, dtype: int64

Unique values and their counts in column 'medpil2a' when 'medpill' is NaN:
medpil2a
NaN    2681
Name: count, dtype: int64

Unique values and their counts in column 'medpil3a' when 'medpill' is NaN:
medpil3a
NaN    2681
Name: count, dtype: int64

Unique values and their counts in column 'medpil4a' when 'medpill' is NaN:
medpil4a
NaN    2681
Name: count, dtype: int64

--------------------------------------------------
Unique values and their counts in column 'medpil1b' when 'medpill' is NaN:
medpil1b
NaN    2681
Name: count, dtype: int64

Unique values and their counts in column 'medpil2b' when 'medpill' is NaN:
medpil2b
NaN    2681
Name: count, dtype: int64

Unique values and their counts in column 'medpil3b' when 'medpill' is NaN:
medpil3b
NaN    2681
Name: count, dtype: int64

Unique values and their counts in column 'medpil4b' when 'medpill' is NaN:
medpil4b
NaN    2681
Name: count, dtype: int64

____________________________________________________________________________________________________

Cbiroth

In [ ]:
#so lets check the unique values in each of those
col_to_process = ['cntrybir', 'cbiroth']


# Iterate through the specified columns and print unique values and their counts
for column in col_to_process:
    unique_values = data_cbiroth[column].unique()
    value_counts = data_cbiroth[column].value_counts(dropna=False)
    print(f"Unique values in column '{column}':")
    print(unique_values)
    print("\n")
    print(f"Value counts in column '{column}':")
    print(value_counts)
    print()


Unique values in column 'cntrybir':
['Yes', 'No']
Categories (2, object): ['Yes' < 'No']


Value counts in column 'cntrybir':
cntrybir
Yes    2727
No        3
Name: count, dtype: int64

Unique values in column 'cbiroth':
['' '1' 'Dubai, UAE']


Value counts in column 'cbiroth':
cbiroth
              2727
1                2
Dubai, UAE       1
Name: count, dtype: int64

In [ ]:
import numpy as np
import pandas as pd

# Assuming data_cbiroth dataframe is already present

# Step 1: If cntrybir is "Yes", replace specific entries in cbiroth with "in_india"
replace_entries_yes = ['', '.', '9', 'Nepal', 'NEPAL', 'sultanpur', 'Sultanpur']
data_cbiroth.loc[(data_cbiroth['cntrybir'] == 'Yes') & 
                 (data_cbiroth['cbiroth'].isin(replace_entries_yes)), 'cbiroth'] = "in_india"

# data_cbiroth.loc[(data_cbiroth['cntrybir'] == 'Yes') & 
#                  (data_cbiroth['cbiroth'].isin(replace_entries_yes)), 'cbiroth'] = np.nan



# # Step 1: If cntrybir is "Yes", replace specific entries in cbiroth with "in_india"
# data_cbiroth.loc[(data_cbiroth['cntrybir'] == 'Yes') & 
#                  (data_cbiroth['cbiroth'].isin(replace_entries_yes)), 'cbiroth'] = "in_india"

# Step 2: If cntrybir is "No", replace certain entries in cbiroth
replace_entries_no = ['9', '', '.', 'india', 'INDIA', '1', 'India', 'Jaipur', 'JAIPUR', 'BEHAR', 'KSA', 'DELHI']
data_cbiroth.loc[(data_cbiroth['cntrybir'] == 'No') & 
                 (data_cbiroth['cbiroth'].isin(replace_entries_no)), 'cbiroth'] = 'outside_india'

# Replace specific entries with mapped values
replace_mapping = {
    'Nepal': 'nepal',
    'UAE': 'uae',
    'usa': 'usa',
    'USA':'usa',
    'CANADA': 'canada',
    'UK': 'uk',
    'pakistan': 'pakistan',  # keep as is
    'DUBAI': 'uae', 
    'dubai': 'uae',
    'Dubai': 'uae',
    '(Dubai, UAE)': 'uae',
    'Dubai, UAE' : 'uae',
    'DUBAI (UAE)' : 'uae',
}

data_cbiroth['cbiroth'] = data_cbiroth['cbiroth'].replace(replace_mapping)

# Step 3: If cntrybir is NaN, replace specific entries in cbiroth with "unknown_country"
replace_entries_nan = ['9', '', '.', 'sultanpur']
data_cbiroth.loc[(data_cbiroth['cntrybir'].isna()) & 
                 (data_cbiroth['cbiroth'].isin(replace_entries_nan)), 'cbiroth'] = "unknown_country"

cntrybir  cbiroth</br>
</br>
Yes       in_india         2727</br>
</br>
No        outside_india       2</br>
</br>
          uae                 1</br>
</br>
Name: count, dtype: int64

# creating a target column which is an 'OR' operation </br> of ['asthdoc', 'asthmaev']

In [ ]:
target = ['asthdoc', 'asthmaev']

# Iterate over each column in category_cols
for col in target:
    print(f"Unique values and counts for column '{col}':")
    # Get unique values and their counts
    value_counts = data_cate[col].value_counts(dropna=False)
    print(value_counts)
    print()  # Blank line for readability
# Check for missing values
    missing_values = data_cate[col].isnull().sum()
    print(f"Number of missing values in '{col}' column: {missing_values}")
    print()

    # Print data type of the column
    print(f"Data type of '{col}' column: {data_cate[col].dtype}")
    print("_"*40)
    print()

Unique values and counts for column 'asthdoc':</br>
asthdoc</br>
NaN    2683</br>
Yes      37</br>
No       10</br>
Name: count, dtype: int64</br>

Number of missing values in 'asthdoc' column: 2683</br>
</br>
Data type of 'asthdoc' column: category</br>
________________________________________</br>
</br>
Unique values and counts for column 'asthmaev':</br>
asthmaev</br>
No     2683</br>
Yes      47</br>
Name: count, dtype: int64</br>
</br>
Number of missing values in 'asthmaev' column: 0</br>
</br>
Data type of 'asthmaev' column: category</br>
________________________________________</br>

In [ ]:
# Filter rows where asthdoc is 'Yes' and asthmaev is either 'No' or NaN
cases = data_cate[(data_cate['asthdoc'] == 'Yes') & 
                  ((data_cate['asthmaev'] == 'No') | (data_cate['asthmaev'].isna()))]

# Display the filtered rows
print(f"Number of such cases: {len(cases)}")
print(cases)

Number of such cases: 0

In [ ]:
asthdoc	asthmaev	asthma_combined
  Yes	  No	        Yes
  No	  Yes	        Yes
  NaN	  No	        No
  NaN	  NaN	       None

In [ ]:
# Create a new column as the OR of 'asthdoc' and 'asthmaev'
data_cate['asthma_combined'] = data_cate[['asthdoc', 'asthmaev']].apply(
    lambda row: 'Yes' if 'Yes' in row.values else ('No' if 'No' in row.values else None), axis=1
)

print(data_cate['asthma_combined'].value_counts(dropna=False))

# Drop the target columns
data_cate.drop(columns=['asthdoc', 'asthmaev'], inplace=True)

# Verify the changes
#print(data_cate.head())
print(data_cate.columns)

# END OF PRE-PROCESSING